<a href="https://colab.research.google.com/github/jfrausto7/Quill/blob/main/llama3_2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
!pip uninstall uno-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastVisionModel
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Mllama vision patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers = False,
    finetune_language_layers = True,
    fientune_attention_modules = True,
    finetune_mlp_modules = True,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rsolar = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.language_model` require gradients


In [ ]:
!pip install datasets
from datasets import load_dataset

# Hugging face dataset here
dataset = load_dataset("dataset")

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

1040_tax.png:   0%|          | 0.00/112k [00:00<?, ?B/s]

estimated_tax.png:   0%|          | 0.00/112k [00:00<?, ?B/s]

life_insurance.png:   0%|          | 0.00/96.1k [00:00<?, ?B/s]

disability_insurance.png:   0%|          | 0.00/102k [00:00<?, ?B/s]

...

wedding_venue.png:   0%|          | 0.00/105k [00:00<?, ?B/s]

volunteer_signup.png:   0%|          | 0.00/108k [00:00<?, ?B/s]

vehicle_registration.png:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

w2_tax.png:   0%|          | 0.00/110k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
instruction = "You are a form filling assistant. You are tasked to identify the blanks in the form images and identify if the information I need to provide is found on a physical form or online. As well, before you give me the blank fields, your response should ask a question to the user like 'For this [form type] form, you'll need to provide:' and end your response with a question asking if the user needs an explanation about what a certain blank field is.

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["caption"]} ]
        },
    ]
    return { "messages" : conversation }

In [ ]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

In [ ]:
FastVisionModel.for_trining(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_caollator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    )
)

In [ ]:
trainer_stats = trainer.train()